In [ ]:
import os

os.chdir("..")

In [ ]:
import geopandas as gpd
from src.inserts import data_inserts 
import polars as pl
import requests
import sqlite3
import duckdb
di = data_inserts()

In [ ]:
# di.insert_regions()
# di.insert_states()
# di.insert_divisions()
# di.insert_county()
# di.insert_track()
# di.insert_datasets()
# di.insert_years()
# di.insert_geo_full()